# Notes

Different problems give different number of points: 2, 3 or 4.

Please, fill `STUDENT` variable with your name, so that we call collect the results automatically. Each problem contains specific validation details. We will do our best to review your assignments, but please keep in mind, that for this assignment automatic grade (between $0$ an $1$) is the primary source of ground truth.

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)

In [ ]:
import numpy as np
import torch

In [ ]:
STUDENT = "Gleb Ivashkevich"
ASSIGNMENT = 3
TEST = False

In [ ]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 11

# Basic PyTorch

### 1. ReLU activation (2 points).

ReLU is the most commonly used activation function in many deep learning application. It's defined as

$$
ReLU(x) = \max(0, x).
$$

Outpu must be of the same shape as input, and **will be tested against three random combinations of input array dimensions ($100 \leq n < 1000 $)**, while values of the input are drawn from standard normal distribution. Number of dimensions of the input will also be selected randomly and is either 1, 2 or 3.

In [ ]:
def relu(arr):
    t = arr.clone()
    t[t<0] = 0.
    return t

In [ ]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, relu)

### 2. Mean squared error (2 points).

In this problem you need to calculate MSE for a pair of tensors `y_true` and `y_pred`. MSE is defined as usual:

$$
L_{MSE} = \frac{1}{N} \sum_i \left(y_i - \hat y_i\right)^2
$$

Note, however, that `y_true` and `y_pred`may be of **different shape**. While `y_true` is always $(N,)$, `y_pred` may be $(N,1)$, $(1, N)$ or $(N,)$. Input values are drawn from standard normal distribution and **shape is selected randomly ($100 \leq n < 1000 $)**.

In [ ]:
def mse(y_true, y_pred):
    return (y_true - y_pred.flatten()).pow(2).mean()

In [ ]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, mse)

### 3. Character-level encoding (4 points).

In computations in general and in machine learning specifically letters cannot be used directly, as computers only know aboun numbers. Text data may be encoded in many different ways in natural language processing tasks.

One of the simplest ways to encode letters is to use one-hot encoded representation, with letters being "class labels". A letter is represented by a tensor of shape $(26,)$.

Then, for example, word "python" would be transformed into a tensor of shape $(6, 26)$ with all elements being $0$, except $(0, 15)\sim p,\,(1, 24)\sim y,\,(2, 19)\sim t,...$ being $1$. A phrase would be represented with 3-dimensional tensor.

In this problem you need to create a tensor, which represents a list of words `words` of length $N$. The only characters used are those from `string.ascii_lowercase`, and words are of different length $L_i$. Output must be of shape $(N, \max(L_i), 26)$.

Dimension 0 corresponds to words themselves, with `tensor[0]` being a represetation of `words[0]`. Note, that you need to use padding: although trivial in this case, you must remember, that tensor must accomodate for a longest word, thus dimension 1 is $\max(L_i)$.

Note also, that the only loop you need here is a loop over `words`, there's no need to loop over the resulting tensor.

The result will be tested against three predefined lists of word, with all words being lowercase and containing only ASCII characters.

In [ ]:
import string

def encode(words):
    """Basic version of encode."""

    shape = (len(words), np.max([len(w) for w in words]), 26)
    encoding = torch.zeros(*shape)

    for wi, w in enumerate(words):
        char_idx = [string.ascii_lowercase.find(e) for e in w]
        pos_idx = np.arange(len(w))
        encoding[wi, pos_idx, char_idx] = 1
    return encoding

def encode_numpy(words):
    """Basic version of encode."""

    shape = (len(words), np.max([len(w) for w in words]), 26)
    encoding = torch.zeros(*shape)

    for wi, w in enumerate(words):
        char_idx = np.frombuffer(w.encode(), dtype=np.int8) - 97
        pos_idx = np.arange(len(w))
        encoding[wi, pos_idx, char_idx] = 1
    return encoding

In [ ]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, encode)

# Gradients

### 4. Generate gradient (1 point).

For 2-dimensional tensor `tr`, calculate a gradient of $\sum\log tr_{ij}$. Note, that you're provided with dimensions and interval, and not the `tr` inself:

- `dims` is a tuple, so that `tr.size()` equals `dims`,
- `lims` is an interval, so that elements of `tr` are integeres, uniformly sampled from `[lims[0], lims[1])` interval (note, that lims[1] is **not** included).

Result must be a tensor of the same shape as `tr` (namely, `dims`), containing gradients of the following function:

$$\sum_{ij}\log tr_{ij}.$$

and generated `tr` itself.

Result **will be tested against multiple random combinations of input tensor dimensions ($10 \leq n < 100 $) and sampling interval (`lims[0]=1`, `10 <= lims[1] < 100`)**.

**Hint**: think on how you can validate the solution yourself, given that gradient can be computed manually on paper.

In [ ]:
def generate_grad(dims, lims):
    """Generate gradient of `log(x)`."""
    # Creating original tensor
    tr = torch.randint(lims[0], lims[1], dims,
                       requires_grad=True,
                       dtype=torch.float)

    # Calculating scalar function and gradient
    c = tr.log().sum()
    c.backward()
    return tr.grad, tr

#### Comment and review

Gradient for the function we use can be calculated analytically:

$$
\frac{\partial}{\partial x_{km}}\sum_{ij}\log x_{ij}
= \sum_{ij} \frac{\partial}{\partial x_{km}} \log x_{ij}
= \sum_{ij} \frac{\delta_{ik}\delta_{jm}}{x_{ij}}
= \frac{1}{x_{km}}
$$

We can now easily check, if gradient is correct by modifying the function we use:

In [ ]:
gr, tr = generate_grad((1, 50), (1, 20))

Now `tr` contains tensor, which was generated inside the function, and `gr` contains the corresponding gradient of the scalar function. It's easy to check, if calculation inside the function is correct:

In [ ]:
assert np.allclose(gr.numpy(), 1/tr.detach().numpy())

In [ ]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, generate_grad)

### 5. Find a minimum (2 points, manually graded).

Consider the following scalar function:

$$
f(x) = ax^2 + bx + c
$$

Given the $a,b,c$, find $x$, which minimizes $f(x)$. Note this:

- $a,b,c$ are fixed, and generated in such a way, that minimum always exists ($f(x)$ is convex),
- $x$ is a scalar value, i.e. 0-dimensional tensor.

**Hint:** stop for a second to think about which tensor must contain `requires_grad` and on which scalar should `.backward()` be called.

In [ ]:
def generate_coeffs():
    a = torch.rand(size=()) * 10
    b = -10 + torch.rand(size=()) * 10
    c = -10 + torch.rand(size=()) * 10
    return a, b, c

def func(x, a, b, c):
    return x.pow(2) * a + x * b + c

In [ ]:
EPOCHS = 20
LR = 1e-1
DELTA = 1e-2
STARTING_VAL = 51.  # Consider choosing different starting values for x to speed-up the optimization

# Initialize x, a, b, c
x = torch.tensor(STARTING_VAL, requires_grad=True)
a, b, c = generate_coeffs()

# Collect f(x) values during training for visuzalition later on
f_vals = []

for i in range(EPOCHS):
    current_f = func(x, a, b, c)
    current_f.backward()
    
    f_vals.append(current_f.item())

    with torch.no_grad():
        x.sub_(LR * x.grad)
    x.grad.zero_()

    if i % 20 == 0:
        print(f"Epoch {i}: loss = {f_vals[-1]}")
    
    # For the first iteration f_vals[-2] is non-existent
    # Brutally catching this with try ... except
    try:
        if f_vals[-2] - f_vals[-1] < DELTA:
            break
    except:
        pass

In [ ]:
if f_vals:
    grid = np.linspace(-5, 5, 100)
    plt.plot(grid, np.square(grid) * a.item() + grid * b.item() + c.item())
    plt.hlines(f_vals[-1], -5, 5, "firebrick", "--", linewidth=1)

In [ ]:
PROBLEM_ID = 5
GRADE = 0

if TEST:
    total_grade += GRADE

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")